:::{note} Tutorial 9. **Server**
:icon: false

#### Effectively working with the server

The Panel server can be launched either from the commandline (using `panel serve`) or programmatically (using `panel.serve()`). In this guide we will discover how to run and configure server instances using these two options.

:::

In [ ]:
import panel as pn

pn.extension()

In [ ]:
panel serve app.py

In [ ]:
pn.serve({'/': '})

In [ ]:
md = pn.pane.Markdown('ABC')
md

In [ ]:
md._models

In [ ]:
1. Browser makes request -> Server creates a session (and executes app.py) and serves the template
2. When browser renders the HTML page includes JS that opens Websocket connection
3. Server receives WS open request -> sends Document JSON representation across Websocket
4. Rendered

5. pn.state.on_session_destroyed(...)



class A():
    pass

pn.state.cache[pn.state.curdoc] = A

In [ ]:
--setup 
pn.state.on_session_create()
pn.state.schedule_task()
pn.state.on_session_destroyed()

## Bokeh Server & Sessions

The Bokeh server is built on Tornado, which handles all of the communication between the browser and the backend. Whenever a user accesses the app or dashboard in a browser a new session is created which executes the app code and creates a new ``Document`` containing the models served to the browser where they are rendered by BokehJS.

<div style="margin-left: auto; margin-right: auto; display: block">
<img src="https://bokeh.pydata.org/en/latest/_images/bokeh_serve.svg"></img>
</div>

Each server can serve multiple sessions and Panel will make the current session state available to your application via the `pn.state` object. When the application is first run or when a callback runs in the context of a session the `pn.state.curdoc` object will reflect the current Bokeh `Document`:

In [ ]:
pn.state.cache

In [ ]:
pn.state

In [ ]:
def app():
    # pn.state
    pn.state.cache

    # pn.session_state
    pn.state.location
    pn.state.curdoc
    pn.state.user

    
    
    return pn.widgets.FloatSlider()

In [ ]:
pn.serve({'/main': app})

In [ ]:
panel serve panel.py --num-threads 4

In [ ]:
pn.config.nthreads = 4

In [ ]:
slider = pn.widgets.FloatSlider()

async def cb(event):
    
    
slider.param.watch(cb, 'value')

slider

In [ ]:
docs[0]

In [ ]:
pn.state.curdoc

## Commandline

Once the app is ready for deployment it can be served using the Bokeh server.  For a detailed breakdown of the design and functionality of Bokeh server, see the [Bokeh documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/server.html). The most important thing to know is that Panel (and Bokeh) provide a CLI command to serve a Python script, app directory, or Jupyter notebook containing a Bokeh or Panel app. To launch a server using the CLI, simply run:

    panel serve app.ipynb

Alternatively you can also list multiple apps:

    panel serve app1.py app2.ipynb

or even serve a number of apps at once:

    panel serve apps/*.py

For development it can be particularly helpful to use the ``--autoreload`` option to `panel serve` as that will automatically reload the page whenever the application code or any of its imports change.


# Launching a server dynamically

The CLI `panel serve` command described below is usually the best approach for deploying applications. However when working on the REPL or embedding a Panel/Bokeh server in another application it is sometimes useful to dynamically launch a server, either using the `.show` method or using the `pn.serve` function.

## `pn.state`

The `pn.state` object makes a whole range of session information available to your application.

:::{note} Attributes
:class: dropdown

`access_token`
: The access token issued by the OAuth provider to authorize requests to its APIs.

`busy`
: A boolean value to indicate whether a callback is being actively processed.

`cache`
: A global cache which can be used to share data between different processes.

`cookies`
: HTTP request cookies for the current session.

`curdoc`
: When running a server session this property holds the current bokeh Document.

`location`
: In a server context this provides read and write access to the URL:
   * `hash`: hash in window.location e.g. '#interact'
   * `pathname`: pathname in window.location e.g. '/user_guide/Interact.html'
   * `search`: search in window.location e.g. '?color=blue'
   * `reload`: Reloads the page when the location is updated.
   * `href` (readonly): The full url, e.g. 'https://localhost:80?color=blue#interact'
   * `hostname` (readonly): hostname in window.location e.g. 'panel.holoviz.org'
   * `protocol` (readonly): protocol in window.location e.g. 'http:' or 'https:'
   * `port` (readonly): port in window.location e.g. '80'

`headers`
: HTTP request headers for the current session.

`refresh_token`
: The refresh token issued by the OAuth provider to authorize requests to its APIs (if available these are usually longer lived than the `access_token`).

`served`
:Whether we are currently inside a script or notebook that is being served using `panel serve`.

`session_args`
: When running a server session this return the request arguments.

`session_info`
: A dictionary tracking information about server sessions:
   * `total` (int): The total number of sessions that have been opened
   * `live` (int): The current number of live sessions
   * `sessions` (dict(str, dict)): A dictionary of session information:
       * `started`: Timestamp when the session was started
       * `rendered`: Timestamp when the session was rendered
       * `ended`: Timestamp when the session was ended
       * `user_agent`: User-Agent header of client that opened the session

`webdriver`
: Caches the current webdriver to speed up export of bokeh models to PNGs.
:::

:::{note} Methods
:class: dropdown

`add_periodic_callback`
: Schedules a periodic callback to be run at an interval set by the period

`as_cached`
: Allows caching data across sessions by memoizing on the provided key and keyword arguments to the provided function.

`cancel_scheduled`
: Cancel a scheduled task by name.

`execute`
: Executes both synchronous and asynchronous callbacks appropriately depending on the context the application is running in.

`kill_all_servers`
: Stops all running server sessions.

`onload`
: Allows defining a callback which is run when a server is fully loaded

`schedule`
: Schedule a callback periodically at a specific time (click [here](../how_to/callbacks/schedule.md) for more details)

`sync_busy`
: Sync an indicator with a boolean value parameter to the busy property on state

:::

## Serving static files

Whether you're launching your application using `panel serve` from the commandline or using `pn.serve` in a script you can also serve static files. When using `panel serve` you can use the `--static-dirs` argument to specify a list of static directories to serve along with their routes, e.g.:

    panel serve some_script.py --static-dirs assets=./assets

This will serve the `./assets` directory on the servers `/assets` route. Note however that the `/static` route is reserved internally by Panel.

Similarly when using `pn.serve` or `panel_obj.show` the static routes may be defined as a dictionary, e.g. the equivalent to the example would be:

    pn.serve(panel_obj, static_dirs={'assets': './assets'})


## Session callbacks

Whenever a request is made to an endpoint that is serving a Panel application a new session is created. If you have to perform some setup or tear down tasks on session creation (e.g. logging) you can define `on_session_created` and `on_session_destroyed` callbacks.


## Periodic callbacks

Periodic callbacks allow periodically updating your application with new data. Below we will create a simple Bokeh plot and display it with Panel:


In [ ]:
import numpy as np
import panel as pn

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure


source = ColumnDataSource({"x": np.arange(10), "y": np.arange(10)})
p = figure()
p.line(x="x", y="y", source=source)

bokeh_pane = pn.pane.Bokeh(p)
bokeh_pane.servable()

Now we will define a callback that updates the data on the `ColumnDataSource` and use the `pn.state.add_periodic_callback` method to schedule updates every 200 ms. We will also set a timeout of 5 seconds after which the callback will automatically stop.

```{warning}
The dynamic callbacks may not function properly in the online documentation. Please use them in a notebook or script.
```

In [ ]:
def update():
    data = np.random.randint(0, 2 ** 31, 10)
    source.data.update({"y": data})
    bokeh_pane.param.trigger('object') # Only needed in notebook

cb = pn.state.add_periodic_callback(update, 200, timeout=5000)

In a notebook or bokeh server context we should now see the plot update periodically. The other nice thing about this is that `pn.state.add_periodic_callback` returns `PeriodicCallback` we can call `.stop()` and `.start()` on if we want to stop or pause the periodic execution. Additionally we can also dynamically adjust the period by setting the `timeout` parameter to speed up or slow down the callback.

## Scheduling Tasks

The `pn.state.schedule_task` functionality allows scheduling global tasks at certain times or on a specific schedule. Global tasks are useful for performing periodic actions like updating cached data, performing cleanup actions or other housekeeping tasks, while periodic callbacks should be reserved for making periodic updates to an application.

The different contexts in which global tasks and periodic callbacks run also has implications on how they should be scheduled. Scheduled task **must not** be declared in the application code itself, i.e. if you are serving `panel serve app.py` the callback you are scheduling must not be declared in the `app.py`. It must be defined in an external module or in a separate script declared as part of the `panel serve` invocation using the `--setup` commandline argument.

Scheduling using `pn.state.schedule_task` is idempotent, i.e. if a callback has already been scheduled under the same name subsequent calls will have no effect. By default the starting time is immediate but may be overridden with the `at` keyword argument. The period may be declared using the `period` argument or a cron expression (which requires the `croniter` library). Note that the `at` time should be in local time but if a callable is provided it must return a UTC time. If `croniter` is installed a `cron` expression can be provided using the `cron` argument.

Here is a simple example of a task scheduled at a fixed interval:


In [ ]:
import panel as pn
import datetime as dt
import asyncio

async def task():
    print(f'Task executed at: {dt.datetime.now()}')

pn.state.schedule_task('task', task, period='1s')
await asyncio.sleep(3)

pn.state.cancel_task('task')